# 0 - collection des données


Au lieu d'utiliser l'interface de stackoverflow, il est possible de récuperer directement dans un dataframe avec des requetes python sur l'api afin de dépasser la limite de 50 000 rows, puis d'exporter pour traitement et modélisation.

In [1]:
import requests
import time
import pandas as pd

#resoudre captcha sur site avant et recup id sinon error 500 'https://data.stackexchange.com/stackoverflow/query/new'
def run_query_stackoverflow(sql_query, id_captcha):

    url = f'https://data.stackexchange.com/query/save/1/{id_captcha}'
    data = {"title": "P05", "description": "", "sql" : sql_query, "g-recaptcha-response" : ""}

    response = requests.post(url, data)
    job_id = response.json().get('job_id')
    
    if(job_id is not None) :
        url = f'https://data.stackexchange.com/query/job/{job_id}'
        time.sleep(3)
        response = requests.get(url)
    
    data = response.json().get('resultSets')[0].get('rows')
    columns = [ col['name'] for col in response.json().get('resultSets')[0].get('columns')]
    
    df = pd.DataFrame(data , columns=columns)

    return df

In [2]:
periode = 300000
max_id = 5*periode+1
reqs = []
for i,j in zip(range(1,max_id,periode),range(periode,max_id,periode)):
    reqs.append(f"SELECT Id,Body,Title,Tags FROM posts WHERE Id BETWEEN {i} AND {j} AND ParentId IS NULL")

In [3]:
for req in reqs:
    print(req)

SELECT Id,Body,Title,Tags FROM posts WHERE Id BETWEEN 1 AND 300000 AND ParentId IS NULL
SELECT Id,Body,Title,Tags FROM posts WHERE Id BETWEEN 300001 AND 600000 AND ParentId IS NULL
SELECT Id,Body,Title,Tags FROM posts WHERE Id BETWEEN 600001 AND 900000 AND ParentId IS NULL
SELECT Id,Body,Title,Tags FROM posts WHERE Id BETWEEN 900001 AND 1200000 AND ParentId IS NULL
SELECT Id,Body,Title,Tags FROM posts WHERE Id BETWEEN 1200001 AND 1500000 AND ParentId IS NULL


In [4]:
id_captcha = 1391113
responses = []
for req in reqs:
    responses.append(run_query_stackoverflow(req, id_captcha=id_captcha))

In [5]:
len(responses)

5

In [6]:
df_posts = pd.concat(responses)

In [7]:
df_posts.shape

(240602, 4)

In [8]:
df_posts.to_csv("data/df_posts.csv", index=False)